# Module 1 Data Analysis: Gravity

In this assignment, you will process (reduce) some gravity data, graph it to visualize trends and consider differences among gravity anomalies, and build a few models to explain the data.

To run a block of code, press CTRL+ENTER. Remember that you'll need to re-run a block after you've made a modification to see what the effect is of your change! You'll want to start by running the blocks in sequential order.

In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import altair as alt

In [ ]:
# Common constants
G = 6.67E-11 # Gravitational constant

# Common corrections
def g_lambda(latitude):
    return 978031.8*(1+(0.0053024*(np.sin(np.radians(latitude)))**2)-(0.0000059*np.sin(np.radians(2*latitude)))**2)
def delta_g_eotvos(v, azimuth, latitude):
    return ((4.040*v*np.sin(np.radians(azimuth))*np.cos(np.radians(latitude)))+(0.001211*v**2))
def delta_g_FA(h):
    return 0.3086*h
def delta_g_B(h, density):
    return 2*np.pi*G*density*h

# Common models
def delta_g_sphere(density_difference, r, h, x):
    return 4*np.pi*G*density_difference*(r**3)*h/(3*(((x**2)+(h**2))**(3/2)))
def delta_g_cylinder(density_difference, r, h, x):
    return 2*np.pi*G*density_difference*(r**2)*h/((x**2)+(h**2))
def delta_g_half_sheet(density_difference, t, h, x):
    return 2*G*density_difference*t*((np.pi/2)+np.arctan(x/h))

In [ ]:
# Read in gravity data from Washington DNR surveys
gravity_data=pd.read_csv("2021-06-23_TGEOS_495_Kitsap_Gravity.csv") # Read in the data (as a spreadsheet)
print (gravity_data.head()) # Show the first few lines of the spreadsheet

In [ ]:
# Calculate one free-air anomaly and see if it matches what DNR lists here
lam = gravity_data.loc[0,'LATITUDE_D']     # Greek letter lambda (λ) stands for latitude... but don't call it "lambda" because that's a Python command! 
h = gravity_data.loc[0,'ELEVATION_']       # Store elevation data in the space in memory (a.k.a. variable) called h
predicted=g_lambda(lam)-delta_g_FA(h)      # Predictions are the acceleration due to gravity on a rotating Earth minus a correction because of elevation
observed=gravity_data.loc[0,'OBSERVED_G']  # Store the observed gravity data in a variable we'll call observed
print("Free-air anomaly for station ",gravity_data.loc[0,'OBSERVATIO'],":",observed-predicted)

### Try this:
* In the example above, I looked at the free-air anomaly for one data point. Can you change which data point we look at in the above block of code. Note that all lists in Python have an index (that is, row numbers) that start at zero.
* How do the Bouguer anomalies compare with what you can calculate?

In [ ]:
# Make a graph with observed gravity on the Y axis and transect distance on the X axis
observed_graph = alt.Chart(gravity_data).mark_point(color='black').encode(
    x=alt.X('Transect_D:Q', axis=alt.Axis(title="Distance along Transect (m)")),
    y=alt.Y('OBSERVED_G:Q', scale=alt.Scale(zero=False), axis=alt.Axis(title="Observed Gravity (mGal)"))
)

display(observed_graph)

### Try this:
* Copy the block of code above, and modify it to make a new graph, called FA_graph. Use it to graph the free-air anomaly. 
* Do the same with Complete Bouguer anomaly.
* You can put two graphs on the same axes by using the "+" symbol between the graphs' names. Try that with the FA and Bouguer graph. You can put them one above another using "&" and side-by-side using "|".

In [ ]:
# Make a graph of the predicted gravity anomaly due to a half-sheet starting at 20,000 m
# Choose whatever density difference, thickness, and depth you like. 
# First we need to predict g due to a half sheet! 
# Let's make our predictions every 100m from 0 to 45 km along the transect
predictions=pd.DataFrame(columns=['Position','delta_g'])
predictions['Position'] = np.linspace(0,45000,100)
predictions['delta_g'] = delta_g_half_sheet(density_difference=1000, t=10000, h=1000, x=predictions['Position']-20000)
print(predictions.head()) # Print out a few values to check

# Now make a graph of the predictions
predicted_graph = alt.Chart(predictions).mark_line(color='red').encode(
    x=alt.X('Position:Q', axis=alt.Axis(title="Distance along Transect (m)")),
    y=alt.Y('delta_g:Q', scale=alt.Scale(zero=False), axis=alt.Axis(title="Predicted Gravity (m/s^2)"))
)

display(predicted_graph)

In [ ]:
display(predicted_graph + observed_graph)

### Try This!

How would you fix the graphs of the observed and predicted gravity so that the trends in both could be seen? 

### And the final bit...

Can you build a model that fits the general shape and scale of the gravity anomaly data? Which set of observations did you use, and why? Is your model the only one that fits? How does the model compare to other information about the area at [the WA DNR Geologic Map pa

In [ ]:
# Easter egg for the problem set...
latitude = 47.2529
azimuth_plane = 270
azimuth_ship=0
v_ship=22
v_plane=800
h_ship=0
h_plane=10000
print('ship:',g_lambda(latitude)+delta_g_eotvos(v_ship,azimuth_ship,latitude)+0.3086*h_ship)
print('plane:',g_lambda(latitude)+delta_g_eotvos(v_plane,azimuth_plane,latitude)+0.3086*h_plane)